In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import rasters as rt
from solar_apparent_time import calculate_solar_day_of_year, calculate_solar_hour_of_day
from verma_net_radiation import verma_net_radiation, verma_net_radiation_table, daily_Rn_integration_verma
from ECOv002_calval_tables import load_combined_eco_flux_ec_filtered, load_metadata_ebc_filt, load_calval_table

In [2]:
repo_root = os.path.dirname(os.getcwd())
package_dir = os.path.join(repo_root, 'verma_net_radiation')
generated_input_table_filename = os.path.join(package_dir, "ECOv002-cal-val-PT-JPL-SM-inputs.csv")
generated_output_table_filename = os.path.join(package_dir, "ECOv002-cal-val-PT-JPL-SM-outputs.csv")

In [3]:
generated_input_table_filename

'/Users/halverso/Projects/verma-net-radiation/verma_net_radiation/ECOv002-cal-val-PT-JPL-SM-inputs.csv'

In [4]:
generated_output_table_filename

'/Users/halverso/Projects/verma-net-radiation/verma_net_radiation/ECOv002-cal-val-PT-JPL-SM-outputs.csv'

In [5]:
model_inputs_gdf = load_calval_table()
model_inputs_gdf

,Unnamed: 0,ID,vegetation,climate,STICinst,BESSinst,MOD16inst,PTJPLSMinst,ETinst,ETinstUncertainty,...,MAP,StartDate,EndDate,LE_count,closure_ratio,geometry,time_UTC,ST_K,ST_C,Ta_C
0,0,US-NC3,ENF,Cfa,270.345200,78.53355,392.851840,307.021970,487.383423,118.916280,...,1320.00,10/1/18 05:00,1/1/22 05:00,9576,1.02,POINT (-76.656 35.799),2019-10-02 19:09:40,305.10,31.95,32.658920
1,1,US-Mi3,CVM,Dfb,232.141600,229.20093,640.118470,375.089300,106.825577,167.919460,...,1012.70,10/1/18 05:00,12/28/19 04:00,12170,0.92,POINT (-80.637 41.8222),2019-06-23 18:17:17,304.34,31.19,24.227982
2,2,US-Mi3,CVM,Dfb,356.355740,335.23154,625.661700,284.686250,NaN,132.936340,...,1012.70,10/1/18 05:00,12/28/19 04:00,12170,0.92,POINT (-80.637 41.8222),2019-06-27 16:35:42,304.06,30.91,26.178862
3,3,US-Mi3,CVM,Dfb,332.938400,326.68680,624.254330,251.414490,178.827545,141.132420,...,1012.70,10/1/18 05:00,12/28/19 04:00,12170,0.92,POINT (-80.637 41.8222),2019-06-30 15:44:10,301.80,28.65,22.527096
4,4,US-Mi3,CVM,Dfb,286.854030,237.21654,511.082180,228.520170,154.791626,114.809410,...,1012.70,10/1/18 05:00,12/28/19 04:00,12170,0.92,POINT (-80.637 41.8222),2019-07-01 14:53:48,303.18,30.03,23.280691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060,1060,US-xAE,GRA,Cfa,70.923310,172.37459,81.645230,15.282976,NaN,56.385185,...,778.85,10/1/18 06:00,12/1/22 00:00,29615,0.60,POINT (-99.0588 35.4106),2021-12-11 16:01:12,278.78,5.63,3.815752
1061,1061,US-xAE,GRA,Cfa,116.543190,121.81641,65.469320,22.186659,NaN,40.509410,...,778.85,10/1/18 06:00,12/1/22 00:00,29615,0.60,POINT (-99.0588 35.4106),2022-03-25 22:45:31,293.28,20.13,19.266186
1062,1062,US-xAE,GRA,Cfa,129.880100,0.00000,118.777240,55.343586,NaN,52.403820,...,778.85,10/1/18 06:00,12/1/22 00:00,29615,0.60,POINT (-99.0588 35.4106),2022-04-12 22:53:09,301.94,28.79,32.110336
1063,1063,US-xAE,GRA,Cfa,2.707851,140.38632,126.490524,40.434025,NaN,57.769722,...,778.85,10/1/18 06:00,12/1/22 00:00,29615,0.60,POINT (-99.0588 35.4106),2022-04-14 14:45:37,290.72,17.57,10.464681


In [6]:
model_inputs_gdf.columns

Index(['Unnamed: 0', 'ID', 'vegetation', 'climate', 'STICinst', 'BESSinst',
       'MOD16inst', 'PTJPLSMinst', 'ETinst', 'ETinstUncertainty', 'PET', 'Rn',
       'ESI', 'RH', 'Ta', 'LST', 'SM', 'NDVI', 'NDVI-UQ', 'albedo',
       'albedo-UQ', 'LST_err', 'view_zenith', 'Rg', 'EmisWB', 'time_utc',
       'solar_time', 'solar_hour', 'local_time', 'LE', 'LE_filt', 'LEcorr25',
       'LEcorr50', 'LEcorr75', 'LEcorr_ann', 'H_filt', 'Hcorr25', 'Hcorr50',
       'Hcorr75', 'Hcorr_ann', 'NETRAD_filt', 'G_filt', 'SM_surf', 'SM_rz',
       'AirTempC', 'SW_IN', 'RH_percentage', 'ESIrn_STIC', 'ESIrn_PTJPLSM',
       'ESIrn_MOD16', 'ESIrn_BESS', 'ESIrn_Unc_ECO', 'ESIrn_LEcorr50', 'JET',
       'eco_time_utc', 'Site Name', 'Date-Time', 'Site ID', 'Name', 'Lat',
       'Long', 'Elev', 'Clim', 'Veg', 'MAT', 'MAP', 'StartDate', 'EndDate',
       'LE_count', 'closure_ratio', 'geometry', 'time_UTC', 'ST_K', 'ST_C',
       'Ta_C'],
      dtype='object')

In [7]:
ST_K = np.array(model_inputs_gdf.LST)
model_inputs_gdf["ST_K"] = ST_K
print(np.nanmin(ST_K), np.nanmean(ST_K), np.nanmax(ST_K))

258.72 302.1800469295775 359.26


In [8]:
ST_C = ST_K - 273.15
model_inputs_gdf["ST_C"] = ST_C
print(np.nanmin(ST_C), np.nanmean(ST_C), np.nanmax(ST_C))

-14.42999999999995 29.030046929577487 86.11000000000001


In [9]:
NDVI = np.array(model_inputs_gdf.NDVI)
model_inputs_gdf["NDVI"] = NDVI
print(np.nanmin(NDVI), np.nanmean(NDVI), np.nanmax(NDVI))

-0.02429185 0.4528892517239436 0.94546


In [10]:
Ta_C = np.array(model_inputs_gdf.Ta)
model_inputs_gdf["Ta_C"] = Ta_C
print(np.nanmin(Ta_C), np.nanmean(Ta_C), np.nanmax(Ta_C))

-14.605048 22.321587967441317 39.710495


In [11]:
RH = np.array(model_inputs_gdf.RH)
model_inputs_gdf["RH"] = RH
print(np.nanmin(RH), np.nanmean(RH), np.nanmax(RH))

0.27253073 0.42692000418779347 0.98366296


In [12]:
Rg = np.array(model_inputs_gdf.Rg)
model_inputs_gdf["Rg"] = Rg
print(np.nanmin(Rg), np.nanmean(Rg), np.nanmax(Rg))

-23.763361 606.9121518676056 1042.9371


In [13]:
Rn = np.array(model_inputs_gdf.Rn)
model_inputs_gdf["Rn"] = Rn
print(np.nanmin(Rn), np.nanmean(Rn), np.nanmax(Rn))

0.0 414.2791522507042 763.1359


In [14]:
albedo = np.array(model_inputs_gdf.albedo)
model_inputs_gdf["albedo"] = albedo
print(np.nanmin(albedo), np.nanmean(albedo), np.nanmax(albedo))

0.015407953 0.10903688726384977 0.6173986


In [15]:
SM = np.array(model_inputs_gdf.SM)
model_inputs_gdf["SM"] = SM
print(np.nanmin(SM), np.nanmean(SM), np.nanmax(SM))

0.0 0.16790158060469482 0.89711


In [16]:
EmisWB = np.array(model_inputs_gdf.EmisWB)
model_inputs_gdf["EmisWB"] = EmisWB
print(np.nanmin(EmisWB), np.nanmean(EmisWB), np.nanmean(EmisWB))

0.734 0.9682704225821597 0.9682704225821597


In [17]:
model_inputs_gdf

,Unnamed: 0,ID,vegetation,climate,STICinst,BESSinst,MOD16inst,PTJPLSMinst,ETinst,ETinstUncertainty,...,MAP,StartDate,EndDate,LE_count,closure_ratio,geometry,time_UTC,ST_K,ST_C,Ta_C
0,0,US-NC3,ENF,Cfa,270.345200,78.53355,392.851840,307.021970,487.383423,118.916280,...,1320.00,10/1/18 05:00,1/1/22 05:00,9576,1.02,POINT (-76.656 35.799),2019-10-02 19:09:40,305.10,31.95,32.658920
1,1,US-Mi3,CVM,Dfb,232.141600,229.20093,640.118470,375.089300,106.825577,167.919460,...,1012.70,10/1/18 05:00,12/28/19 04:00,12170,0.92,POINT (-80.637 41.8222),2019-06-23 18:17:17,304.34,31.19,24.227982
2,2,US-Mi3,CVM,Dfb,356.355740,335.23154,625.661700,284.686250,NaN,132.936340,...,1012.70,10/1/18 05:00,12/28/19 04:00,12170,0.92,POINT (-80.637 41.8222),2019-06-27 16:35:42,304.06,30.91,26.178862
3,3,US-Mi3,CVM,Dfb,332.938400,326.68680,624.254330,251.414490,178.827545,141.132420,...,1012.70,10/1/18 05:00,12/28/19 04:00,12170,0.92,POINT (-80.637 41.8222),2019-06-30 15:44:10,301.80,28.65,22.527096
4,4,US-Mi3,CVM,Dfb,286.854030,237.21654,511.082180,228.520170,154.791626,114.809410,...,1012.70,10/1/18 05:00,12/28/19 04:00,12170,0.92,POINT (-80.637 41.8222),2019-07-01 14:53:48,303.18,30.03,23.280691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060,1060,US-xAE,GRA,Cfa,70.923310,172.37459,81.645230,15.282976,NaN,56.385185,...,778.85,10/1/18 06:00,12/1/22 00:00,29615,0.60,POINT (-99.0588 35.4106),2021-12-11 16:01:12,278.78,5.63,3.815752
1061,1061,US-xAE,GRA,Cfa,116.543190,121.81641,65.469320,22.186659,NaN,40.509410,...,778.85,10/1/18 06:00,12/1/22 00:00,29615,0.60,POINT (-99.0588 35.4106),2022-03-25 22:45:31,293.28,20.13,19.266186
1062,1062,US-xAE,GRA,Cfa,129.880100,0.00000,118.777240,55.343586,NaN,52.403820,...,778.85,10/1/18 06:00,12/1/22 00:00,29615,0.60,POINT (-99.0588 35.4106),2022-04-12 22:53:09,301.94,28.79,32.110336
1063,1063,US-xAE,GRA,Cfa,2.707851,140.38632,126.490524,40.434025,NaN,57.769722,...,778.85,10/1/18 06:00,12/1/22 00:00,29615,0.60,POINT (-99.0588 35.4106),2022-04-14 14:45:37,290.72,17.57,10.464681


In [18]:
model_inputs_gdf.keys()

Index(['Unnamed: 0', 'ID', 'vegetation', 'climate', 'STICinst', 'BESSinst',
       'MOD16inst', 'PTJPLSMinst', 'ETinst', 'ETinstUncertainty', 'PET', 'Rn',
       'ESI', 'RH', 'Ta', 'LST', 'SM', 'NDVI', 'NDVI-UQ', 'albedo',
       'albedo-UQ', 'LST_err', 'view_zenith', 'Rg', 'EmisWB', 'time_utc',
       'solar_time', 'solar_hour', 'local_time', 'LE', 'LE_filt', 'LEcorr25',
       'LEcorr50', 'LEcorr75', 'LEcorr_ann', 'H_filt', 'Hcorr25', 'Hcorr50',
       'Hcorr75', 'Hcorr_ann', 'NETRAD_filt', 'G_filt', 'SM_surf', 'SM_rz',
       'AirTempC', 'SW_IN', 'RH_percentage', 'ESIrn_STIC', 'ESIrn_PTJPLSM',
       'ESIrn_MOD16', 'ESIrn_BESS', 'ESIrn_Unc_ECO', 'ESIrn_LEcorr50', 'JET',
       'eco_time_utc', 'Site Name', 'Date-Time', 'Site ID', 'Name', 'Lat',
       'Long', 'Elev', 'Clim', 'Veg', 'MAT', 'MAP', 'StartDate', 'EndDate',
       'LE_count', 'closure_ratio', 'geometry', 'time_UTC', 'ST_K', 'ST_C',
       'Ta_C'],
      dtype='object')

In [19]:
model_inputs_gdf.albedo

0       0.215445
1       0.117238
2       0.117280
3       0.084629
4       0.120526
          ...   
1060    0.092853
1061    0.111844
1062    0.106782
1063    0.106775
1064    0.113165
Name: albedo, Length: 1065, dtype: float64

In [20]:
type(model_inputs_gdf)

geopandas.geodataframe.GeoDataFrame

In [21]:
results = verma_net_radiation_table(model_inputs_gdf)
results

[2025-08-12 15:16:26 INFO] variable SWout_Wm2 min: 0.000 mean: 66.193 max: 278.058 nan: 0.00% (nan)
[2025-08-12 15:16:26 INFO] variable SWnet_Wm2 min: 0.000 mean: 540.741 max: 929.140 nan: 0.00% (nan)
[2025-08-12 15:16:26 INFO] variable SWnet_Wm2 min: 0.000 mean: 540.741 max: 929.140 nan: 0.00% (nan)
[2025-08-12 15:16:26 INFO] variable LWin_Wm2 min: 175.737 mean: 343.651 max: 473.521 nan: 0.00% (nan)
[2025-08-12 15:16:26 INFO] variable LWout_Wm2 min: 240.846 mean: 462.295 max: 693.334 nan: 0.00% (nan)
[2025-08-12 15:16:26 INFO] variable Rn_Wm2 min: 0.000 mean: 422.206 max: 843.390 nan: 0.00% (nan)
[2025-08-12 15:16:26 INFO] variable LWin_Wm2 min: 175.737 mean: 343.651 max: 473.521 nan: 0.00% (nan)
[2025-08-12 15:16:26 INFO] variable LWout_Wm2 min: 240.846 mean: 462.295 max: 693.334 nan: 0.00% (nan)
[2025-08-12 15:16:26 INFO] variable Rn_Wm2 min: 0.000 mean: 422.206 max: 843.390 nan: 0.00% (nan)


,Unnamed: 0,ID,vegetation,climate,STICinst,BESSinst,MOD16inst,PTJPLSMinst,ETinst,ETinstUncertainty,...,geometry,time_UTC,ST_K,ST_C,Ta_C,SWout_Wm2,SWnet_Wm2,LWin_Wm2,LWout_Wm2,Rn_Wm2
0,0,US-NC3,ENF,Cfa,270.345200,78.53355,392.851840,307.021970,487.383423,118.916280,...,POINT (-76.656 35.799),2019-10-02 19:09:40,305.10,31.95,32.658920,117.527293,427.983267,433.191382,465.788054,395.386594
1,1,US-Mi3,CVM,Dfb,232.141600,229.20093,640.118470,375.089300,106.825577,167.919460,...,POINT (-80.637 41.8222),2019-06-23 18:17:17,304.34,31.19,24.227982,99.458206,748.885694,355.960473,463.110120,641.736047
2,2,US-Mi3,CVM,Dfb,356.355740,335.23154,625.661700,284.686250,NaN,132.936340,...,POINT (-80.637 41.8222),2019-06-27 16:35:42,304.06,30.91,26.178862,98.376177,740.435423,385.005187,471.101631,654.338979
3,3,US-Mi3,CVM,Dfb,332.938400,326.68680,624.254330,251.414490,178.827545,141.132420,...,POINT (-80.637 41.8222),2019-06-30 15:44:10,301.80,28.65,22.527096,72.080661,779.644139,357.100788,458.191551,678.553376
4,4,US-Mi3,CVM,Dfb,286.854030,237.21654,511.082180,228.520170,154.791626,114.809410,...,POINT (-80.637 41.8222),2019-07-01 14:53:48,303.18,30.03,23.280691,84.676015,617.875585,358.727378,459.922447,516.680516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060,1060,US-xAE,GRA,Cfa,70.923310,172.37459,81.645230,15.282976,NaN,56.385185,...,POINT (-99.0588 35.4106),2021-12-11 16:01:12,278.78,5.63,3.815752,26.634684,260.211916,237.686968,335.647726,162.251159
1061,1061,US-xAE,GRA,Cfa,116.543190,121.81641,65.469320,22.186659,NaN,40.509410,...,POINT (-99.0588 35.4106),2022-03-25 22:45:31,293.28,20.13,19.266186,32.532471,258.341529,317.465236,409.440488,166.366277
1062,1062,US-xAE,GRA,Cfa,129.880100,0.00000,118.777240,55.343586,NaN,52.403820,...,POINT (-99.0588 35.4106),2022-04-12 22:53:09,301.94,28.79,32.110336,37.608161,314.587139,401.910116,459.984925,256.512330
1063,1063,US-xAE,GRA,Cfa,2.707851,140.38632,126.490524,40.434025,NaN,57.769722,...,POINT (-99.0588 35.4106),2022-04-14 14:45:37,290.72,17.57,10.464681,44.918145,375.760655,266.094367,395.330786,246.524236


In [22]:
time_UTC = model_inputs_gdf.time_UTC
time_UTC

0       2019-10-02 19:09:40
1       2019-06-23 18:17:17
2       2019-06-27 16:35:42
3       2019-06-30 15:44:10
4       2019-07-01 14:53:48
               ...         
1060    2021-12-11 16:01:12
1061    2022-03-25 22:45:31
1062    2022-04-12 22:53:09
1063    2022-04-14 14:45:37
1064    2022-04-20 19:38:47
Name: time_UTC, Length: 1065, dtype: object

In [23]:
geometry = model_inputs_gdf.geometry
geometry

0         POINT (-76.656 35.799)
1        POINT (-80.637 41.8222)
2        POINT (-80.637 41.8222)
3        POINT (-80.637 41.8222)
4        POINT (-80.637 41.8222)
                  ...           
1060    POINT (-99.0588 35.4106)
1061    POINT (-99.0588 35.4106)
1062    POINT (-99.0588 35.4106)
1063    POINT (-99.0588 35.4106)
1064    POINT (-99.0588 35.4106)
Name: geometry, Length: 1065, dtype: geometry

In [24]:
day_of_year = calculate_solar_day_of_year(
    time_UTC=time_UTC,
    geometry=geometry
)

day_of_year

array([[275, 275, 275, ..., 275, 275, 275],
       [174, 174, 174, ..., 174, 174, 174],
       [178, 178, 178, ..., 178, 178, 178],
       ...,
       [102, 102, 102, ..., 102, 102, 102],
       [104, 104, 104, ..., 104, 104, 104],
       [110, 110, 110, ..., 110, 110, 110]],
      shape=(1065, 1065), dtype=int32)

In [25]:
daily_Rn_integration_verma(
    Rn_Wm2=results["Rn_Wm2"],
    time_UTC=results["time_UTC"],
    geometry=results["geometry"]
)

ValueError: Data must be 1-dimensional, got ndarray of shape (1065, 1065) instead

In [ ]:
model_inputs_gdf.to_csv(generated_input_table_filename, index=False)

In [ ]:
results.to_csv(generated_output_table_filename, index=False)